# setup

In [1]:
import os

if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('..')

In [2]:
import numpy as np
import h5py

from tqdm import tqdm

In [3]:
from data_loader.util import load_label_files, load_challenge_data, resample, slide_and_cut_beat_aligned

# read

In [4]:
# Define the weights, the SNOMED CT code for the normal class, and equivalent SNOMED CT codes.
weights_file = 'weights.csv'
normal_class = '426783006'
equivalent_classes = [['713427006', '59118001'], ['284470004', '63593006'], ['427172004', '17338001']]

In [5]:
input_directory_label = '/home/josegfer/datasets/challenge2020/data'
label_dir = '/home/josegfer/datasets/challenge2020/data'
# Find the label files.
print('Finding label and output files...')
label_files = load_label_files(input_directory_label)

Finding label and output files...


# loop

In [6]:
num_files = len(label_files)
# num_files = 5
# recordings2save = []
# ratio2save = []
erros = []

In [ ]:
for i in tqdm(range(num_files)):
    # print('{}/{}'.format(i + 1, num_files))
    recording, header, name = load_challenge_data(label_files[i], label_dir)
    recording[np.isnan(recording)] = 0

    # divide ADC_gain and resample
    recording = resample(recording, header, 500)

    # slide and cut
    try:
        recording, info2save = slide_and_cut_beat_aligned(recording, 1, 5000, 500,
                                                      seg_with_r=False, beat_length=400)
    except:
        print('skipping file: {}, idx: {}'.format(name, i))
        erros.append(name)
        continue

In [9]:
len(erros), erros

(32,
 ['E03910',
  'E04290',
  'E05353',
  'E06575',
  'E06885',
  'E10058',
  'HR01334',
  'HR02441',
  'HR02550',
  'HR03076',
  'HR03567',
  'HR04323',
  'HR05434',
  'HR07331',
  'HR07385',
  'HR09602',
  'HR10827',
  'HR12700',
  'HR12735',
  'HR13591',
  'HR14697',
  'HR14894',
  'HR15280',
  'HR16202',
  'HR16787',
  'HR17451',
  'HR17732',
  'HR18611',
  'HR19000',
  'HR20409',
  'HR20778',
  'HR21694'])

In [ ]:
n_lead = 12
n_segment = 10
beat_length = 400

# h5f = h5py.File('data/challenge2020.h5', 'w')
# X = h5f.create_dataset('recording', (num_files, n_lead, n_segment, beat_length), dtype='f8')
# r = h5f.create_dataset('ratio', (num_files, 1, n_segment), dtype='f8')

In [ ]:
for i in tqdm(range(num_files)):
    # print('{}/{}'.format(i + 1, num_files))
    recording, header, name = load_challenge_data(label_files[i], label_dir)
    recording[np.isnan(recording)] = 0

    # divide ADC_gain and resample
    recording = resample(recording, header, 500)

    # slide and cut
    try:
        recording, info2save = slide_and_cut_beat_aligned(recording, 1, 5000, 500,
                                                      seg_with_r=False, beat_length=400)
        X[i, :, :, :] = np.transpose(recording, (0, 2, 1, 3))
        r[i, :, :] = info2save
        # recordings2save[i, :, :, :] = np.transpose(recording, (0, 2, 1, 3))
        # ratio2save[i, :, :] = info2save
        # recordings2save.append(recording[0])
        # ratio2save.append(info2save)
    except:
        print('skipping file: {}, idx: {}'.format(name, i))
        error.append(name)
        # recordings2save[i, :, :, :] = np.zeros(shape = (1, n_lead, n_segment, beat_length))
        # ratio2save[i, :, :] = np.zeros(shape = (1, 1, n_segment))
        continue
recordings2save = np.array(recordings2save)
recordings2save = np.transpose(recordings2save, (0, 2, 1, 3))
ratio2save = np.array(ratio2save)

# write

In [8]:
# h5f.close()